In [1]:
import numpy as np
import pandas as pd
import redis

In [2]:
#connection to database
#Connect to redis client

hostname='redis-19868.c267.us-east-1-4.ec2.cloud.redislabs.com'
portnumber=19868
password='ElZFvkWqGrcayZt7SMmnmknmc9rX5nUC'

r=redis.StrictRedis(host=hostname,
                    port=portnumber,
                    password=password)


In [3]:
r.ping()

True

In [4]:
r.keys()

[]

In [5]:
#Load the data

In [5]:
np_file=np.load('dataframe_students.npz',allow_pickle=True)
x_values=np_file['arr_0']
col_names=np_file['arr_1']

df=pd.DataFrame(x_values,columns=col_names)
df

,Name,Roll,Facial Features
0,Aadish Jain,1001,"[1.4249021, 0.5167993, 1.1065452, -0.39924377,..."
1,Abhinav Gangil,1002,"[2.5445485, 1.1459275, -0.2571401, -0.9233791,..."
2,Aditi Wekhande,1003,"[-0.15211493, 0.6594273, 0.94268084, 1.4843241..."
3,Aditya Kshitiz,1004,"[1.6784743, 0.84842515, 0.44553462, -0.7783499..."
4,Aditya Mastud,1005,"[-0.4629953, -0.071729705, -0.083244145, -0.42..."
...,...,...,...
75,Vedant Upadhyay,2081,"[0.68810403, -0.13327494, 0.58295095, 1.208180..."
76,Vijit Balsori,1079,"[0.83771884, 1.1717737, 0.025990069, -0.092498..."
77,Vineet Verma,1080,"[1.4692615, 2.0162005, 0.15505144, 0.8998549, ..."
78,Votte Sriyans,1081,"[0.6702863, -0.64987075, -0.3088921, 1.2300224..."


In [6]:
df['name_roll']=df['Name']+'@'+df['Roll']
df

,Name,Roll,Facial Features,name_roll
0,Aadish Jain,1001,"[1.4249021, 0.5167993, 1.1065452, -0.39924377,...",Aadish Jain@1001
1,Abhinav Gangil,1002,"[2.5445485, 1.1459275, -0.2571401, -0.9233791,...",Abhinav Gangil@1002
2,Aditi Wekhande,1003,"[-0.15211493, 0.6594273, 0.94268084, 1.4843241...",Aditi Wekhande@1003
3,Aditya Kshitiz,1004,"[1.6784743, 0.84842515, 0.44553462, -0.7783499...",Aditya Kshitiz@1004
4,Aditya Mastud,1005,"[-0.4629953, -0.071729705, -0.083244145, -0.42...",Aditya Mastud@1005
...,...,...,...,...
75,Vedant Upadhyay,2081,"[0.68810403, -0.13327494, 0.58295095, 1.208180...",Vedant Upadhyay@2081
76,Vijit Balsori,1079,"[0.83771884, 1.1717737, 0.025990069, -0.092498...",Vijit Balsori@1079
77,Vineet Verma,1080,"[1.4692615, 2.0162005, 0.15505144, 0.8998549, ...",Vineet Verma@1080
78,Votte Sriyans,1081,"[0.6702863, -0.64987075, -0.3088921, 1.2300224...",Votte Sriyans@1081


In [7]:
records=df[['name_roll','Facial Features']].to_dict(orient='records')
len(records)

80

In [8]:
#save data in redis
#use redis hashes

In [9]:
for record in records:
    name_roll=record['name_roll']
    vector=record['Facial Features']

    #convert numpy array to bytes
    vector_bytes=vector.tobytes()
    r.hset(name='class:register',key=name_roll,value=vector_bytes)